In [1]:
import os
import json
import numpy as np
import networkx as nx
import geopandas as gpd
from tqdm import tqdm
from pathlib import Path
import pickle

# === Parameters ===
GT_FILE = "../traffic_volume/GT_AADT_8years.json"
GPKL_FILE = "../highway_network/uk_driving_graph_simplified_travel_time_added.gpickle"
NODE_LSOA_FILE = "../node_features/node_to_lsoa.json"
SUBGRAPH_DIR = "subgraphs"
TIME_THRESHOLD_HOURS = 1
TIME_THRESHOLD_SEC = TIME_THRESHOLD_HOURS * 3600

os.makedirs(SUBGRAPH_DIR, exist_ok=True)

# === Load data ===
print("📦 Loading data...")
with open(GT_FILE) as f:
    gt = json.load(f)

with open(NODE_LSOA_FILE) as f:
    node_to_lsoa = json.load(f)

with open(GPKL_FILE, "rb") as f:
    G = pickle.load(f)


📦 Loading data...


In [2]:
from pathlib import Path
import heapq
import numpy as np
import json
from tqdm import tqdm
import networkx as nx
import shutil

# === Competitive O/D wavefront: single PQ, first-claim wins, no cross-traversal through claimed nodes ===
def competitive_partition(G, u, v, cutoff_sec):
    INF = float("inf")
    # distances, parents, winners
    dist = {"O": {u: 0.0}, "D": {v: 0.0}}
    parent = {"O": {u: None}, "D": {v: None}}
    winner = {}  # node -> "O"/"D"

    # min-heap over (time, side, node)
    pq = [(0.0, "O", u), (0.0, "D", v)]
    heapq.heapify(pq)

    # helper: yield directed neighbors + edge cost for a side
    def neighbors(side, x):
        if side == "O":
            # traverse IN-edges (reverse direction)
            # skip the ego edge both directions by node-pair check
            for a, _, k, data in G.in_edges(x, keys=True, data=True):
                if (a == u and x == v) or (a == v and x == u):
                    continue
                w = data.get("travel_time")
                if w is None or w <= 0:
                    continue
                yield a, float(w)
        else:
            # side == "D": traverse OUT-edges (forward)
            for _, b, k, data in G.out_edges(x, keys=True, data=True):
                if (x == u and b == v) or (x == v and b == u):
                    continue
                w = data.get("travel_time")
                if w is None or w <= 0:
                    continue
                yield b, float(w)

    while pq:
        t, side, x = heapq.heappop(pq)
        # stale?
        if t != dist[side].get(x, float("inf")):
            continue
        # if other side already claimed x, skip (no cross through claimed nodes)
        if x in winner:
            continue
        # claim x
        winner[x] = side
        if t > cutoff_sec:
            continue
        # relax neighbors for this side only
        for y, w in neighbors(side, x):
            new_t = t + w
            if new_t < dist[side].get(y, INF) and new_t <= cutoff_sec:
                dist[side][y] = new_t
                parent[side][y] = x
                # only push if y not already claimed by the other side
                if y not in winner:
                    heapq.heappush(pq, (new_t, side, y))

    return dist, parent, winner  # per-side times/parents, and final winning label for each node

In [3]:
# === Main processing (drop-in) ===
for edge_id_str in tqdm(gt.keys(), desc="Processing GT-labelled edges"):
    u_str, v_str, k_str = edge_id_str.split("_")
    u, v, k = int(u_str), int(v_str), int(k_str)

    ego_dir = Path(SUBGRAPH_DIR) / edge_id_str
    meta_path = ego_dir / "meta.json"

    if meta_path.exists():
        continue
    elif ego_dir.exists():
        shutil.rmtree(ego_dir)

    # Run competitive expansion (no copies, no views)
    dist, parent, winner = competitive_partition(G, u, v, TIME_THRESHOLD_SEC)

    # Build meta
    meta = {}
    G_rev = G.reverse(copy=False)

    for node_id, side in winner.items():
        if str(node_id) not in node_to_lsoa or node_id in (u, v):
            continue
        t = dist[side].get(node_id, None)
        if t is None or not np.isfinite(t):
            continue

        meta[str(node_id)] = {
            "direction": side,
            "travel_time_sec": float(t)
        }

    with open(meta_path, "w") as f:
        json.dump(meta, f, indent=2)

Processing GT-labelled edges: 100%|██████████| 5088/5088 [2:04:19<00:00,  1.47s/it]  
